In [ ]:
import pickle 
import pandas as pd
import spacy
from tqdm.auto import tqdm
from sklearn.metrics.pairwise import cosine_similarity
from tqdm.notebook import tqdm
tqdm.pandas()
import re
import transformers
from sentence_transformers import SentenceTransformer, util
import nltk
from nltk import tokenize
import pickle
import urllib.request as requests
import json
import statistics
import torch
from collections import defaultdict
import copy
import math

In [ ]:

model = SentenceTransformer('all-mpnet-base-v2')

df=pd.read_csv('gmo_to_uniqueId.csv')
materialID=df['lastRunJob'].to_list()

# with open('PreProcessed_segmented.pkl', 'rb') as f:
#     newmetalist=pickle.load(f)

with open('New_PreProcessed_segmented.pkl', 'rb') as f:
    newnewmetalist=pickle.load(f)

sampled_datadf=pd.read_csv("Sampled_Adword_Labeled.csv")

#For Normal process with target labels only

df1=pd.read_csv('AllTiersAdword.csv')
target_dict=dict(zip(df1['Name'], df1.index))

In [ ]:
def clean(text):
    text=text.lower()
    text=text.replace("(", "")
    text=text.replace(")", "")
    text=text.replace(";", "")
    text=text.replace(",", "")
    text=text.replace("+", "")
    text=text.replace(".", "")
    text=text.replace("&", "")
    return text

In [ ]:
df1.head()

In [ ]:
targetlist=df1['Name'].to_list()



In [ ]:
truthdatadf=pd.read_csv('AdwordLabeledDatabase.csv')


In [ ]:
def getadword(episode, truthdatadf):
    for i in range(len(truthdatadf.lastRunJob)):
        if episode == truthdatadf.lastRunJob[i]:
            return truthdatadf.adword[i]

In [ ]:
# temp=targetlistcalc[27]
# temp=temp.replace('*News business use case needs to be scoped by NBCU - priority TBD', '')
# targetlistcalc[27]=temp

In [ ]:
listofepisodes=sampled_datadf['lastRunJob'].to_list()

In [ ]:
#listofepisodes=['FILE_MAF_20220211T205601Z_GMO_00000000001313_01', 'FILE_MAF_20220211T205607Z_GMO_00000000001534_01', 'FILE_MAF_20210727T231031Z_GMO_00000000001577_01' ]

In [ ]:
def keydict(dataframe, targetlist):
    d=defaultdict(list)
    for target in targetlist:
        try:

            minidf=dataframe.loc[dataframe['adword']==target]
            minilist=minidf['lastRunJob'].to_list()
            d[target]=minilist
        except:
            continue
    
    return d



In [ ]:
targetkey_filelist=keydict(sampled_datadf, targetlist)


In [ ]:
# #NRS bulk with top 1 save to file regular
# dict_to_save=defaultdict(list)
# for target in tqdm(targetlist):
#     scores=[]
#     for ep in tqdm(listofepisodes):
#         index=materialID.index(ep)
#         groundtruth=getadword(ep,truthdatadf)
#         # if groundtruth in target_dict:
#         #         x=target_dict[groundtruth]
#         x=target_dict[target]
#         #print(groundtruth)
#         print(x)
#         segmentscore=[]
#         for i in range(len(newmetalist[index])):
#             if len(newmetalist[index][i])==0:
#                 continue
#             emb1=model.encode(newmetalist[index][i]) #, convert_to_tensor=True)
#             emb2=model.encode(targetlist) #, convert_to_tensor=True)
#             cos_sim=util.cos_sim(emb1,emb2)

#             #For OG Class
#             MaxAggregation = {}
#             for i in range(cos_sim.shape[1]):
#                 all_sentence_combinations=[]
#                 for j in range(cos_sim.shape[0]):
#                     all_sentence_combinations.append([cos_sim[j][i], j, i])
#                 all_sentence_combinations = sorted(all_sentence_combinations, key=lambda x: x[0], reverse=True)
#                 MaxAggregation[i]=all_sentence_combinations[0]
                

#             segmentscore.append(MaxAggregation[x][0])

            
                    
#         avg=(sum(segmentscore)/len(segmentscore))
#         scores.append(avg)
        
#     print(scores)
#     dict_to_save[target]=scores

In [ ]:
# with open('NRS_top1_regular.pkl', 'wb') as f:
#     pickle.dump(dict_to_save, f)


In [ ]:
# with open('NRS_top1_regular.pkl', 'rb') as f:
#     NRS_dict=pickle.load(f)


In [ ]:
def get_key(val):
    for key, value in target_dict.items():
         if val == value:
             return key
 
    return "key doesn't exist"

In [ ]:
dad=[]
for i in range(len(newnewmetalist)):
    mom=[]
    for j in range(len(newnewmetalist[i])):
        if len(newnewmetalist[i][j])==0:
            continue
        mom.append(newnewmetalist[i][j])
    dad.append(mom)

In [ ]:
print(targetkey_filelist)

In [ ]:
#Relevance calculation per episode top1
dictofrelevance=copy.deepcopy(targetkey_filelist)
for key, value in tqdm(targetkey_filelist.items()): #for each IAB Label
    if len(targetkey_filelist[key])==0:
        continue
    for i in tqdm(range(len(targetkey_filelist[key]))): #for each File
        index=materialID.index(targetkey_filelist[key][i])
        x=target_dict[key] #Index of the target label
        MaxAggregation=defaultdict(list)

        for l in range(len(dad[index])): #For each Segment
            # print(len(newmetalist[index]))
            if len(dad[index][l])==0: 
                continue
            emb1=model.encode(dad[index][l]) #, convert_to_tensor=True)
            emb2=model.encode(targetlist) #, convert_to_tensor=True)
            cos_sim=util.cos_sim(emb1,emb2)

        #Add all pairs to a list with their cosine similarity score
        
            for k in range(cos_sim.shape[1]):
                all_sentence_combinations=[]
                for j in range(cos_sim.shape[0]):
                    all_sentence_combinations.append([cos_sim[j][k], j, k])
                all_sentence_combinations = sorted(all_sentence_combinations, key=lambda x: x[0], reverse=True)
                MaxAggregation[k].append(all_sentence_combinations[0])
    
     
        dictofrelevance[key][i]={dictofrelevance[key][i]:MaxAggregation}
     

with open('Relevance_Top1_Tiers.pkl', 'wb') as f:
    pickle.dump(dictofrelevance, f)   
 
        #RS and NRS for OG Class

        # RSlist=MaxAggregation[x]
        # score=[]
        # for s in RSlist:
        #     score.append(s[0])

        # rs=sum(score)/len(score)

        # NRSlist=NRS_dict[key]
        # nrsdenom=sum(NRSlist)-rs/len(NRSlist)

        # NRS=rs/nrsdenom

        
        # #RS and ClassPredicted for Predicted Class

        # order=[]
        # for key in MaxAggregation.keys():
        #     l=MaxAggregation[key]
        #     summ=[]
        #     for val in l:
        #         summ.append(val[0])
        #     order.append(sum(summ))
        
        
        # max_val=max(order)
        # max_index=order.index(max_val)
        # ClassPredicted=get_key(max_index)

        # RSlistss=MaxAggregation[max_index]
        # scoress=[]
        # for s in RSlistss:
        #     scoress.append(s[0])

        # RS_CP=sum(scoress)/len(scoress)

        # d={targetkey_filelist[key][i]:{'NRS': NRS, "Class_Predicted": ClassPredicted, "RS_CP": RS_CP, "RS_OG": rs }}

        


        


        

    

In [ ]:
#Relevance calculation per episode top3
dictofrelevance=copy.deepcopy(targetkey_filelist)
for key, value in tqdm(targetkey_filelist.items()): #for each IAB Label
    if len(targetkey_filelist[key])==0:
        continue
    for i in tqdm(range(len(targetkey_filelist[key]))): #for each File
        index=materialID.index(targetkey_filelist[key][i])
        x=target_dict[key] #Index of the target label
        MaxAggregation=defaultdict(list)

        for l in range(len(newmetalist[index])): #For each Segment
            # print(len(newmetalist[index]))
            if len(newmetalist[index][l])==0: 
                continue
            emb1=model.encode(newmetalist[index][l]) #, convert_to_tensor=True)
            emb2=model.encode(targetlist) #, convert_to_tensor=True)
            cos_sim=util.cos_sim(emb1,emb2)

        #Add all pairs to a list with their cosine similarity score
        
            for k in range(cos_sim.shape[1]):
                all_sentence_combinations=[]
                for j in range(cos_sim.shape[0]):
                    all_sentence_combinations.append([cos_sim[j][k], j, k])
                all_sentence_combinations = sorted(all_sentence_combinations, key=lambda x: x[0], reverse=True)
                templist=all_sentence_combinations[0:3]
                temp=[]
                sent=[]
                for item in templist:
                    temp.append(item[0])
                    sent.append(list([item[1],item[2]]))
                avg=(sum(temp)/len(temp))
                comb=list([avg, sent])
                MaxAggregation[k].append(comb)
            
            # print(MaxAggregation)
            
    
        # print(MaxAggregation)
        # print(dictofrelevance[key][i])
        dictofrelevance[key][i]={dictofrelevance[key][i]:MaxAggregation}
        # print(dictofrelevance[key][i])

with open('Relevance_AdditionalSignal_top3_regular.pkl', 'wb') as f:
    pickle.dump(dictofrelevance, f)  

In [ ]:
#Relevance calculation per episode Harmonic
dictofrelevance=copy.deepcopy(targetkey_filelist)
for key, value in tqdm(targetkey_filelist.items()): #for each IAB Label
    if len(targetkey_filelist[key])==0:
        continue
    for i in tqdm(range(len(targetkey_filelist[key]))): #for each File
        index=materialID.index(targetkey_filelist[key][i])
        x=target_dict[key] #Index of the target label
        MaxAggregation=defaultdict(list)

        for l in range(len(newmetalist[index])): #For each Segment
            # print(len(newmetalist[index]))
            if len(newmetalist[index][l])==0: 
                continue
            emb1=model.encode(newmetalist[index][l]) #, convert_to_tensor=True)
            emb2=model.encode(targetlist) #, convert_to_tensor=True)
            cos_sim=util.cos_sim(emb1,emb2)

        #Add all pairs to a list with their cosine similarity score
        
            for k in range(cos_sim.shape[1]):
                all_sentence_combinations=[]
                for j in range(cos_sim.shape[0]):
                    all_sentence_combinations.append([cos_sim[j][k], j, k])
                all_sentence_combinations = sorted(all_sentence_combinations, key=lambda x: x[0], reverse=True)
                all_sentence_combinations = [ele for ele in all_sentence_combinations if ele[0] > 0]
                templist=all_sentence_combinations
                temp=[]
                sent=[]
                for item in templist:
                    temp.append(float(item[0]))
                try:
                    sent.append([templist[0][1],templist[0][2],templist[0][0]])
                    avg=statistics.harmonic_mean(temp)
                    comb=list([avg, sent])
                    MaxAggregation[k].append(comb)
                except:
                    sent.append(['-'])
                    avg=0
                    comb=list([avg, sent])
                    MaxAggregation[k].append(comb)



            
            # print(MaxAggregation)
            
    
        # print(MaxAggregation)
        # print(dictofrelevance[key][i])
        dictofrelevance[key][i]={dictofrelevance[key][i]:MaxAggregation}
        # print(dictofrelevance[key][i])

with open('Relevance_Harmonic_regular.pkl', 'wb') as f:
    pickle.dump(dictofrelevance, f)  

In [ ]:
with open('Relevance_Top1_Tiers.pkl', 'rb') as f:
     Relevance_dict=pickle.load(f)

In [28]:
Relevance_dict['Sports'][0]['FILE_MAF_20210731T020535Z_GMO_00000000040741_01'][200]

[]

In [19]:
print(target_dict)

{'Automotive': 0, 'Books and Literature': 1, 'Business and Finance': 2, 'Education': 3, 'Events and Attractions': 4, 'Family and Relationships': 5, 'Fine Art': 6, 'Food & Drink': 7, 'Healthy Living': 8, 'Hobbies & Interests': 9, 'Home & Garden': 10, 'Medical Health': 11, 'Movies': 12, 'Music and Audio': 13, 'Pets': 14, 'Pop Culture': 15, 'Real Estate': 16, 'Religion & Spirituality': 17, 'Science': 18, 'Shopping': 19, 'Sports': 20, 'Style & Fashion': 21, 'Technology & Computing': 22, 'Television': 23, 'Travel': 24, 'Video Gaming': 25, 'Auto Body Styles': 26, 'Motorcycles': 27, 'Road-Side Assistance': 28, 'Auto Parts': 29, 'Auto Repair': 30, 'Auto Shows': 31, 'Auto Rentals': 32, 'College Education': 33, 'Primary Education': 34, 'Fashion Events': 35, 'Malls & Shopping Centers': 36, 'Museums & Galleries': 37, 'Outdoor Activities': 38, 'Parks & Nature': 39, 'Awards Shows': 40, 'Personal Celebrations & Life Events': 41, 'Religious Events': 42, 'Theater Venues and Events': 43, 'Zoos & Aquariu

In [36]:
def topksentences(dict):
    d=defaultdict(list)
    adwords=[]
    texts=[]
    scores=[]
    for adword in tqdm(target_dict.keys()):
        scoreforfiles=[]
        for k,v in tqdm(dict.items()):#for each label
            if len(dict[k]) == 0:
                continue
            # scoreforfiles=[]
    
            for i in range(len(dict[k])): #for each file in label
                x=target_dict[adword]
                seg_rel=copy.deepcopy(dict[k][i])
                filename=list(seg_rel.keys())[0]
                defdict=seg_rel[filename]
                segments=defdict[x]



                for count,seg in enumerate(segments):
                    seg.append(filename)
                    seg.append(count)                
                    scoreforfiles.append(seg)
        sort=sorted(scoreforfiles, key=lambda x: x[0], reverse=True)
        Final=sort[0:20]
        # print(Final)
        
        for items in Final:
            index=materialID.index(items[3])
            try:
                items.append(dad[index][items[4]][items[1]])
            except Exception as e:
                print(e)
                print(index,items[4],items[1])
                # print(dad[index][items[4]][items[1]])
                print(newnewmetalist[index][items[4]][items[1]])
                # items.append(dad[index][items[4]][items[1]])
                continue
        # print(Final)
        ff=[]
        for f in Final:
            temp=str(f[0])
            temp=temp.replace('tensor(', "")
            temp=temp.replace(')', "")
            adwords.append(adword)
            texts.append(f[5])
            scores.append(temp)
            
            
    
    # d[k].extend(ff)
    d={}
    d['Adword']=adwords
    d['Text']=texts
    d['Score']=scores
    return d
    
        

                
                

In [ ]:
print(materialID.index('FILE_MAF_20210729T074455Z_GMO_00000000044309_01'))

In [37]:
dict_top20=topksentences(Relevance_dict)

  0%|          | 0/160 [00:00<?, ?it/s]

  0%|          | 0/160 [00:00<?, ?it/s]

[[tensor(0.5168), 60, 0, 'FILE_MAF_20211223T065130Z_GMO_00000000002708_01', 0, 'vehicle accident'], [tensor(0.5168), 47, 0, 'FILE_MAF_20211222T230805Z_GMO_00000000273733_01', 0, ' vehicle accident'], [tensor(0.5100), 37, 0, 'FILE_MAF_20210803T024232Z_GMO_00000000346786_01', 4, 'welcome to the royal automobile club'], [tensor(0.5041), 108, 0, 'FILE_MAF_20210824T142522Z_GMO_00000000294142_01', 0, 'um what kind of car are you driving  porsche cayenne'], [tensor(0.5007), 64, 0, 'FILE_MAF_20210803T024232Z_GMO_00000000346786_01', 1, 'so you really are a car man'], [tensor(0.4941), 385, 0, 'FILE_MAF_20220213T011912Z_GMO_00000000022391_01', 0, "mcdonald'’s or burger king hyundai or honda hmm"], [tensor(0.4916), 62, 0, 'FILE_MAF_20211116T134036Z_GMO_00000000341644_01', 3, 'what do you know about cars  i know what i like'], [tensor(0.4660), 81, 0, 'FILE_MAF_20220215T035307Z_GMO_00000000325452_01', 4, " it's what's on your car wheels"], [tensor(0.4653), 182, 0, 'FILE_MAF_20211223T143836Z_GMO_0000

  0%|          | 0/160 [00:00<?, ?it/s]

[[tensor(0.5964), 87, 1, 'FILE_MAF_20220321T225329Z_GMO_00000000354811_01', 1, 'yo all these books'], [tensor(0.5956), 30, 1, 'FILE_MAF_20220214T042858Z_GMO_00000000137235_01', 5, 'and what is this book about yes'], [tensor(0.5863), 39, 1, 'FILE_MAF_20210802T164609Z_GMO_00000000044398_01', 0, " my book of course i don't mind"], [tensor(0.5773), 35, 1, 'FILE_MAF_20210825T010736Z_GMO_00000000324078_01', 0, 'okay so the book'], [tensor(0.5681), 30, 1, 'FILE_MAF_20210729T074455Z_GMO_00000000044309_01', 0, "fletcher's books"], [tensor(0.5654), 20, 1, 'FILE_MAF_20211105T085331Z_GMO_00000000121678_01', 0, ' book club  book club'], [tensor(0.5520), 102, 1, 'FILE_MAF_20220505T091718Z_GMO_00000000354814_01', 4, ' malcolm gladwell colson whitehead and audre lorde'], [tensor(0.5485), 19, 1, 'FILE_MAF_20220213T011914Z_GMO_00000000014814_01', 1, 'i came across this amazing book'], [tensor(0.5392), 8, 1, 'FILE_MAF_20211223T143428Z_GMO_00000000343676_01', 1, 'what book uh this one'], [tensor(0.5349), 

...

  0%|          | 0/160 [00:00<?, ?it/s]

[[tensor(0.5393), 54, 27, 'FILE_MAF_20210802T225204Z_GMO_00000000007468_01', 0, 'michael buy a motorcycle'], [tensor(0.4952), 109, 27, 'FILE_MAF_20220213T120643Z_GMO_00000000038929_01', 2, " they're actually motorized"], [tensor(0.4712), 82, 27, 'FILE_MAF_20210729T235115Z_GMO_00000000346783_01', 1, ' sinderby  the riders are outside'], [tensor(0.4698), 126, 27, 'FILE_MAF_20211223T065125Z_GMO_00000000027759_01', 1, " how'd you get here  my bike"], [tensor(0.4653), 16, 27, 'FILE_MAF_20210809T200440Z_GMO_00000000292342_01', 2, 'what kind of bike you got i used to have a bike'], [tensor(0.4605), 148, 27, 'FILE_MAF_20210727T231238Z_GMO_00000000001594_01', 2, 'we can go upstate on my new motorcycle'], [tensor(0.4568), 34, 27, 'FILE_MAF_20220215T082050Z_GMO_00000000370434_01', 0, 'these  hot wheels'], [tensor(0.4513), 32, 27, 'FILE_MAF_20211222T214941Z_GMO_00000000121162_01', 3, 'mouch how you feeling  like riding a bike lieutenant'], [tensor(0.4493), 57, 27, 'FILE_MAF_20220215T035307Z_GMO_00

  0%|          | 0/160 [00:00<?, ?it/s]

[[tensor(0.4824), 184, 28, 'FILE_MAF_20210730T234421Z_GMO_00000000044076_01', 3, "highway patrol's sending a unit out to pick him up"], [tensor(0.4726), 59, 28, 'FILE_MAF_20220213T184650Z_GMO_00000000047418_01', 1, 'eyes on the road in front of you'], [tensor(0.4726), 59, 28, 'FILE_MAF_20220213T184650Z_GMO_00000000047418_01', 2, 'eyes on the road in front of you'], [tensor(0.4578), 18, 28, 'FILE_MAF_20210802T172724Z_GMO_00000000142561_01', 4, 'vehicle approaching  excuse me'], [tensor(0.4508), 73, 28, 'FILE_MAF_20220213T185405Z_GMO_00000000045315_01', 0, ' drive safely getting back home'], [tensor(0.4479), 39, 28, 'FILE_MAF_20211106T220217Z_GMO_00000000124439_01', 4, "so if this is the vehicle that you used on this path to paradise and this vehicle has now broken down it's abandoned on the side of the road your job is to figure out how to repair this vehicle"], [tensor(0.4441), 17, 28, 'FILE_MAF_20211222T214633Z_GMO_00000000027055_01', 1, 'truck backing up back her up'], [tensor(0.4413

  0%|          | 0/160 [00:00<?, ?it/s]

In [38]:
df11=pd.DataFrame.from_dict(dict_top20)

In [39]:
df11.head()

,Adword,Text,Score
0,Men's Formal Wear,vehicle accident,0.5168
1,Men's Formal Wear,vehicle accident,0.5168
2,Men's Formal Wear,welcome to the royal automobile club,0.5100
3,Men's Formal Wear,um what kind of car are you driving porsche c...,0.5041
4,Men's Formal Wear,so you really are a car man,0.5007


In [40]:
df11.to_csv("Top20Sentences.csv")